In [18]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [19]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [20]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=50
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=20
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))

#FOR EXPECTATION VALUES

#PHI
print("PHI")
print((E_Cj/E_L)**(1.0/4.0)*(a_phi+ad_phi))
phi=np.kron(Id_M,((E_Cj/E_L)**(1.0/4.0))*(a_phi+ad_phi))
print("kron PHI")
print(phi)
phi2=np.kron(Id_M,((E_Cj_2/E_L_2)**(1.0/4.0))*(a_phi+ad_phi))

#COS_THETA
costheta=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
costheta[0][2*M]+=1
costheta[2*M][0]+=1
print(costheta)
print("COS")
cos_theta=np.kron(costheta,Id_N)
print("kron COS")
print(cos_theta)

#N_PHI
N_phi_extended=np.kron(Id_M,N_phi)

#N_theta
n_theta_extended=np.kron(n_theta,Id_N)


#FOR SAVING VALUES OF CALCULATIONS

#1ST VALUES

ng1=np.arange(0.48,0.53,0.04)
print(ng1)
a=np.pi-(np.pi*0.04)
b=np.pi+(np.pi*0.04 + 0.01)
phi_ext1=np.arange(a,b,(0.04*np.pi)*2)
print(phi_ext1)
Energies=[]
Eigenvectors=[]

Expvals_costheta_NG=[]
Expvals_phi_NG=[]
Expvals_N_phi_NG=[]
Expvals_n_theta_NG=[]

Expvals_costheta_PHIEXT=[]
Expvals_phi_PHIEXT=[]
Expvals_N_phi_PHIEXT=[]
Expvals_n_theta_PHIEXT=[]

#2ND VALUES

ng2=np.arange(0.48,0.53,0.04)
phi_ext2=np.arange(a,b,(0.04*np.pi)*2)
Energies2=[]
Eigenvectors2=[]

Expvals2_costheta_NG=[]
Expvals2_phi_NG=[]
Expvals2_N_phi_NG=[]
Expvals2_n_theta_NG=[]

Expvals2_costheta_PHIEXT=[]
Expvals2_phi_PHIEXT=[]
Expvals2_N_phi_PHIEXT=[]
Expvals2_n_theta_PHIEXT=[]


PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
kron PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1

In [21]:
# Construction of the 0-pi Hamiltonian

def Hamiltonian_0pi(EJ,ECS,ECJ,EL,n,phi,alph):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    H1=4*ECS*((n_theta-n*Id_M)@(n_theta-n*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [22]:
def calculate_expval(vect,operator):
    return np.conj(vect)@operator@vect

In [23]:
Vprueba=[1,1,0]
print(Vprueba)
matrix=[[0,1,0],[1,0,1],[0,1,0]]
print(matrix)
a=calculate_expval(Vprueba,matrix)
print(a)


[1, 1, 0]
[[0, 1, 0], [1, 0, 1], [0, 1, 0]]
2


## $\text{1. Plot for } E_{Cs} = E_{Cj} = E_L = 1$, $\varphi_{ext}= \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = i$.

In [24]:
# Calculations 1

for n1 in ng1:
    print("Step for ng = ",n1)
    H1=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,n1,phi_ext_val,alpha)
    print("Hamiltonian built, len(H)=",len(H1)**2)
    #vals,vects=np.linalg.eigh(H)
    valsNG, vectsNG = eigsh(H1, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsNG[0]," and ",valsNG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsNG[:,0]," and ",vectsNG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsNG[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsNG[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsNG[:,0],phi)
    Expval_phi_1=calculate_expval(vectsNG[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsNG[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsNG[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsNG[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsNG[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_NG.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_NG.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_NG.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_NG.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for ng =  0.48
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7049371654884224  and  1.8404210481619834
The eigenvectors of the GS and the 1st excited state are  [ 9.08599572e-19  4.29273316e-18 -4.21396406e-19 ...  1.67367997e-17
 -1.90909994e-17  1.28820003e-16]  and  [ 1.36182481e-19 -7.06418614e-19 -4.87976340e-18 ... -3.07781989e-17
  1.65562856e-17  1.12663253e-17]
Eigenvalues and eigenvectors calculated
Step for ng =  0.52
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.704937165488199  and  1.840421048161806
The eigenvectors of the GS and the 1st excited state are  [ 1.18656372e-19  3.71262288e-18  1.03726789e-17 ...  7.09909868e-18
  1.60675298e-18 -1.07590289e-16]  and  [ 1.28655710e-19  5.42894959e-18  3.74986719e-18 ... -1.41745356e-17
 -2.19346146e-17  4.36495075e-17]
Eigenvalues and eigenvectors calculated


In [25]:
# Calculations 1

for phi1 in phi_ext1:
    print("Step for phi_ext = ",phi1)
    H2=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,ng_val,phi1,alpha)
    print("Hamiltonian built, len(H)=",len(H2)**2)
    #vals,vects=np.linalg.eigh(H)
    valsPHI, vectsPHI = eigsh(H2, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsPHI[0]," and ",valsPHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsPHI[:,0]," and ",vectsPHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsPHI[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsPHI[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsPHI[:,0],phi)
    Expval_phi_1=calculate_expval(vectsPHI[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsPHI[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsPHI[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsPHI[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsPHI[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_PHIEXT.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_PHIEXT.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_PHIEXT.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_PHIEXT.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for phi_ext =  3.015928947446201
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.737229451326558  and  1.807426536609061
The eigenvectors of the GS and the 1st excited state are  [-6.76844651e-18 -2.11591496e-17  1.20005718e-17 ...  1.14399012e-17
 -5.55462184e-17 -1.41806185e-16]  and  [-8.34042350e-18 -1.54623408e-17  1.23213636e-17 ...  1.59626433e-17
  4.90105915e-17  1.96978634e-17]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.2672563597333846
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7372294513267155  and  1.8074265366088729
The eigenvectors of the GS and the 1st excited state are  [ 2.94977937e-18 -8.87993793e-19  1.13246485e-17 ...  9.33974483e-18
  7.11732376e-17 -6.99026739e-17]  and  [ 1.09938901e-18  2.44061838e-19 -8.31358359e-18 ... -5.71455433e-17
 -8.74849097e-17 -1.20983637e-16]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE FIRST SET OF VALUES FOR THE ENERGIES

In [26]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals_costheta_NG)
print("Exp values of phi varying ng are ",Expvals_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [2.884344490716924e-13, -2.5054927369296836e-13, -4.476520662668983e-13, 4.486542161919266e-13]
Exp values of phi varying ng are  [1.6079659030736858e-13, -2.422795254929655e-14, -2.552082038036596e-13, 2.0455139223342126e-13]
Exp values of N_phi varying ng are  [0.07986229011955934, 0.10106746341760232, 0.079862290119555, 0.10106746341757213]
Exp values of n_theta varying ng are  [0.06655789331295803, 0.9120589135131677, 0.9334421066870447, 0.0879410864868042]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0787415241388414, -1.0653339214587125, -1.07874152413883, 1.0653339214586959]
Exp values of phi varying phi_ext are  [0.545523065500263, -0.5707353367712245, -0.5455230655002119, 0.5707353367710997]
Exp values of N_phi varying phi_ext are  [0.0863247591128797, 0.09241049853127131, 0.08632475911286228, 0.09241049853124136]
Exp values of n_theta varying phi_ext are  [0.4999999999999807, 0.5000000000000081, 0.500000000

## $\text{2. Plot for } E_{Cs} = \frac{1}{20}$, $E_{CJ} = 1$, $E_L = \frac{1}{16}$, $\varphi_{ext} = \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = 2i$.

In [27]:
# Calculations 2

for n2 in ng2:
    print("Step for ng = ",n2)
    H3=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,n2,phi_ext_val,alpha2)
    print("Hamiltonian built, len(H)=",len(H3)**2)
    #vals2,vects2=np.linalg.eigh(H2)
    vals2NG, vects2NG = eigsh(H3, k=2, which='SA')
    #vals2,_=np.linalg.eigh(H3)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2NG[0]," and ",vals2NG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2NG[:,0]," and ",vects2NG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2NG[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2NG[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2NG[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2NG[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2NG[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2NG[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2NG[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2NG[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_NG.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_NG.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_NG.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_NG.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for ng =  0.48
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2713101817193545  and  0.2791137889143804
The eigenvectors of the GS and the 1st excited state are  [ 1.02732256e-17  8.37895169e-19  1.83042469e-17 ...  2.07423594e-17
  6.51626423e-17 -9.95848012e-17]  and  [ 5.90571119e-19 -3.89104181e-19 -1.16869101e-17 ...  4.33374430e-17
 -2.69413390e-17 -1.30379773e-16]
Eigenvalues and eigenvectors calculated
Step for ng =  0.52
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2713101817193678  and  0.27911378891438277
The eigenvectors of the GS and the 1st excited state are  [ 1.65035779e-19  9.02563632e-19  2.79611612e-18 ...  4.43081940e-17
  6.42402732e-17 -2.43302489e-16]  and  [-2.79531864e-18 -3.51570114e-18 -7.33111378e-18 ... -1.11879328e-18
  2.52835203e-17  7.38826001e-17]
Eigenvalues and eigenvectors calculated


In [28]:
# Calculations 2

for phi2 in phi_ext2:
    print("Step for phi_ext = ",phi2)
    H4=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi2,alpha2)
    print("Hamiltonian built, len(H)=",len(H4)**2)
    #vals,vects=np.linalg.eigh(H)
    vals2PHI, vects2PHI = eigsh(H4, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2PHI[0]," and ",vals2PHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2PHI[:,0]," and ",vects2PHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2PHI[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2PHI[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2PHI[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2PHI[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2PHI[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2PHI[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2PHI[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2PHI[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_PHIEXT.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_PHIEXT.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_PHIEXT.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_PHIEXT.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for phi_ext =  3.015928947446201
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2734323640219567  and  0.27685723177124916
The eigenvectors of the GS and the 1st excited state are  [ 1.72219469e-17 -2.37308587e-18 -4.92863114e-18 ... -3.53904984e-17
  7.49131668e-18 -2.09721404e-16]  and  [-9.40922010e-18  2.41631682e-18  7.58179413e-18 ... -8.62321333e-17
  8.04580674e-17 -4.58037470e-18]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.2672563597333846
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2734323640219318  and  0.27685723177125615
The eigenvectors of the GS and the 1st excited state are  [-2.73025968e-18 -3.81282873e-18 -4.59828792e-19 ... -1.28107006e-17
 -1.33011314e-18  5.00969411e-17]  and  [ 2.07788525e-18  4.41974531e-18  1.33842124e-17 ... -1.04411211e-16
  6.65325984e-17  2.58508094e-16]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE SECOND SET OF VALUES FOR THE ENERGIES

In [29]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals2_costheta_NG)
print("Exp values of phi varying ng are ",Expvals2_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals2_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals2_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals2_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals2_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals2_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals2_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [6.372263833895028e-13, -6.352119745837051e-13, -1.8179228654223286e-12, 1.8241032411698128e-12]
Exp values of phi varying ng are  [1.2927198060044622e-13, -1.2947227346249904e-13, -3.688566364805042e-13, 3.7534291575307805e-13]
Exp values of N_phi varying ng are  [0.04370134484883785, 0.044600210154699296, 0.04370134484883853, 0.04460021015470089]
Exp values of n_theta varying ng are  [0.011508023424612606, 0.9869239041755583, 0.9884919765753893, 0.013076095824437002]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0891265683419684, -1.0745041784607836, -1.089126568341967, 1.0745041784607066]
Exp values of phi varying phi_ext are  [0.21530721535963937, -0.22023059484259982, -0.21530721535963185, 0.22023059484259686]
Exp values of N_phi varying phi_ext are  [0.043960985809941004, 0.04425921547554168, 0.043960985809939304, 0.04425921547554093]
Exp values of n_theta varying phi_ext are  [0.5000000000060025, 0.499999999993